In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import zipfile
import os
import shutil



In [2]:
df = pd.read_excel('/Users/songtengyu/Desktop/上上7.18订单.xlsx')
df.head(40)

,打印时间：,Unnamed: 1,2024-07-18 11:26:23.680000,Unnamed: 3,Unnamed: 4,Unnamed: 5,页码：第1页，共10页,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,采购订单,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,订单日期：2024-07-18,NaN,NaN,NaN,NaN,订单编号：PO240718040,NaN,NaN,NaN,NaN
3,供方,单位名称,江苏上上电缆集团有限公司,NaN,NaN,联系电话,NaN,13179771777,NaN,NaN
4,NaN,单位地址,溧阳市上上路68号,NaN,NaN,传 真,NaN,无,NaN,NaN
5,NaN,联 系 人,李仁武,NaN,NaN,E_mail,NaN,NaN,NaN,NaN
6,需方,单位名称,德州锦城电装股份有限公司,NaN,NaN,联系电话,NaN,NaN,NaN,NaN
7,NaN,单位地址,中国山东省德州市临邑县城区开元西大街富民路东侧,NaN,NaN,传 真,NaN,NaN,NaN,NaN
8,NaN,联 系 人,李娜,NaN,NaN,E_mail,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def parse_order(df):
    df_all_orders = pd.DataFrame()
    # select order rows between
    # NO	产品编号	描述	规格	供方料号	数量	未税单价	未税金额	含税金额	交货日期
    # and
    # <NA>	<NA>	<NA>	合计	<NA>	<NA>	41814.05	<NA>
    start = None
    end = None
    for i in range(len(df)):
        if df.iloc[i, 0] == 'NO':
            start = i
        if df.iloc[i, 5] == '合计' or df.iloc[i, 0] == '[以下空白]':
            end = i
        if start and end:
            df_info = df.iloc[start:end, :]
            df.columns = df_info.iloc[0, :]
            df_info = df_info[1:]
            df_all_orders = pd.concat([df_all_orders, df_info])
            start = None
            end = None
    return df_all_orders


df_parsed = parse_order(df)
df_parsed.to_csv('dingdan.csv', index=False)